In [1]:
#Importar la libreria para conectar con cassandra en Docker
from cassandra.cluster import Cluster

In [3]:
# Conectarse al contenedor local de Cassandra
cluster = Cluster(['127.0.0.1'])  # usa el puerto 9042 por defecto
session = cluster.connect()

print("Conectado a Cassandra:", session)

Conectado a Cassandra: <cassandra.cluster.Session object at 0x0000015A0DFFD3D0>


In [4]:
#Creacion keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS telecom
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace("telecom")
print("Keyspace telecom listo")

Keyspace telecom listo


In [5]:
#Creacion tabla llamadas
session.execute("""
CREATE TABLE IF NOT EXISTS llamadas (
    numero_origen  TEXT,
    fecha          DATE,
    id_llamada     UUID,
    numero_destino TEXT,
    duracion       INT,
    PRIMARY KEY ((numero_origen), fecha, id_llamada)
);
""")
print("Tabla llamadas creada")

Tabla llamadas creada


In [6]:
import uuid
from datetime import date

In [7]:
#Insertar datos
datos = [
    ("123456789", date(2025, 6, 1), "987654321", 10),
    ("123456789", date(2025, 6, 15), "654987321", 5),
    ("555666777", date(2025, 6, 2), "999888777", 8),
]

for n_origen, f, n_dest, mins in datos:
    session.execute(
        "INSERT INTO llamadas (numero_origen, fecha, id_llamada, numero_destino, duracion) "
        "VALUES (%s, %s, %s, %s, %s)",
        (n_origen, f, uuid.uuid4(), n_dest, mins)
    )
print("Datos insertados")

Datos insertados


In [8]:
#Consulta las llamadas de un número
rows = session.execute("""
SELECT * FROM llamadas
WHERE numero_origen = '123456789';
""")
for row in rows:
    print(row)

Row(numero_origen='123456789', fecha=Date(20240), id_llamada=UUID('589c33ae-6032-4a3c-bc12-1a2261f6d257'), duracion=10, numero_destino='987654321')
Row(numero_origen='123456789', fecha=Date(20254), id_llamada=UUID('dcec0a33-3506-486e-b262-66d99e529156'), duracion=5, numero_destino='654987321')


In [9]:
#llamadas de un número en el rango de fechas
rows = session.execute("""
SELECT * FROM llamadas
WHERE numero_origen = '123456789'
  AND fecha >= '2025-06-01' AND fecha <= '2025-06-30';
""")
for row in rows:
    print(row)

Row(numero_origen='123456789', fecha=Date(20240), id_llamada=UUID('589c33ae-6032-4a3c-bc12-1a2261f6d257'), duracion=10, numero_destino='987654321')
Row(numero_origen='123456789', fecha=Date(20254), id_llamada=UUID('dcec0a33-3506-486e-b262-66d99e529156'), duracion=5, numero_destino='654987321')


In [10]:
#el total de minutos de llamada
rows = session.execute("""
SELECT duracion FROM llamadas
WHERE numero_origen = '123456789'
  AND fecha >= '2025-06-01' AND fecha <= '2025-06-30';
""")
total = sum(r.duracion for r in rows)
print("Minutos consumidos en junio 2025:", total)

Minutos consumidos en junio 2025: 15
